In [1]:
# https://www.kaggle.comcd/code/arjunjoshua/predicting-fraud-in-financial-payment-services/notebook

In [3]:
import os
os.chdir('/Users/samet/projects/imbalanced-data/fraud_detection')

In [4]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
%matplotlib inline

from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
from xgboost.sklearn import XGBClassifier
import src.transformers as trans

warnings.filterwarnings("ignore", category=DeprecationWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
df_train = pd.read_csv('./data/train.csv')
# df_test = pd.read_csv('./data/test.csv')

In [41]:
DEBUG = int(5e5)
TARGET = "isFraud"


df_train = pd.read_csv('./data/train.csv', nrows=DEBUG)
df_train = df_train.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})                        

df_train_model = trans.SelectToModel().transform(df_train).sample(frac=1, random_state=42)

y_train = df_train_model.pop(TARGET)
X_train = df_train_model

# Model

In [9]:
from sklearn.model_selection import cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline, FeatureUnion

DROP_COLUMNS = ['nameOrig', 'nameDest', 'isFlaggedFraud']



transformation_pipeline = Pipeline([
    ('drop_columns', trans.DropColumns(DROP_COLUMNS)),
    ('label_encode', trans.CustomLabelEncoder(['type'])),
    ('amount_vs_old_new_balance_dest', trans.AmountVsOldAndNewBalanceDest()),
    ('amount_vs_old_new_balance_orig', trans.AmountVsOldAndNewBalanceOrig())
])

# weights = np.mean(df_train[TARGET])


# weights = [1, 1/10, 1/25, 1/50, 1/75, 1/99, 1/100, 1/1000]
# param_grid = dict(scale_pos_weight=weights)


weights = sum(df_train[TARGET] == 0) / sum(df_train[TARGET] == 1)

model_pipeline = Pipeline([
    ('xgbclassifier', XGBClassifier(max_depth = 1, scale_pos_weight = weights, n_jobs = 4)) ,
    ])


pipe = Pipeline([
    ('transformation', transformation_pipeline),
    ('model', model_pipeline)
])

# grid = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='roc_auc')


#https://discuss.xgboost.ai/t/how-does-scale-pos-weight-affect-probabilities/1790

In [11]:
pipe.fit(X_train, y_train)
probabilities = pipe.predict_proba(X_test)
print('AUPRC = {}'.format(average_precision_score(y_test, probabilities[:, 1])))

AUPRC = 0.9741079652970728


In [12]:
# from tqdm import tqdm
# for i in tqdm(range(X_test.shape[0])[7000:]):
#     # print(i)
#     probabilities = pipe.predict_proba(X_test.iloc[[i]])
#     if y_test.iloc[i] == 1:
#         print('Fraud!')
#         print(probabilities[0][1]>.5)
#     else:
#         if probabilities[0][1]>.5:
#             print('False Positive!!')

# EDA

In [ ]:
# df_transfer = df.query("isFraud == 1 & type == 'TRANSFER'")
# df_cashout = df.query("isFraud == 1 & type == 'CASH_OUT'")

# len(df_transfer) , len(df_cashout)


In [ ]:
# randomState = 5
# np.random.seed(randomState)
# X_sample = X.sample(frac=.01)

# Y = X['isFraud']
# del X['isFraud']

In [ ]:
# PARAM = 200
# X.loc[(X['isFraud']==0) & (X['step']<PARAM),'step'].hist(bins=PARAM, density=True, alpha=.5)
# X.loc[(X['isFraud']==1) & (X['step']<PARAM),'step'].hist(bins=PARAM, density=True, alpha=.5)

In [ ]:

# sns.catplot(data=X_sample, x="step", y='isFraud', orient='h', hue="type",  aspect=2, alpha=.3 )

In [ ]:
# sns.catplot(data=X_sample, x="amount", y='isFraud', orient='h', hue="type",  aspect=2, alpha=.3 )

In [ ]:
# sns.catplot(data=X_sample, y="errorBalanceDest", x='isFraud', orient='v', hue="type",  aspect=2, alpha=.3, height=5 )